# Many to Many Relationships in SQL

Previously, we have seen one to many relationship.
<img src ="c.png">
Now, we will implement many to many relationship database.
<img src ="d.png">

We will start with a fresh database.

In [1]:
%load_ext sql

In [2]:
import sqlalchemy
sqlalchemy.create_engine('sqlite:///courses.db')

Engine(sqlite:///courses.db)

In [3]:
%sql sqlite:///courses.db 

In [ ]:
%%sql
CREATE TABLE "User" (
	"id"	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
	"name"	TEXT,
	"email"	TEXT
);

In [ ]:
%%sql
CREATE TABLE "Course" (
	"id"	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
	"text"	TEXT
);

In [ ]:
%%sql
CREATE TABLE "Member" (
	"user_id"	INTEGER,
	"course_id"	INTEGER,
	"role"	INTEGER,
	PRIMARY KEY("user_id","course_id")
);

Inserting data into the database.

In [ ]:
%%sql
INSERT into User(name,email) VALUES ('Jane','jane@uvic.ca');
INSERT into User(name,email) VALUES ('Manke','manke@uvic.ca');
INSERT into User(name,email) VALUES ('Ryan','ryan@uvic.ca')

In [4]:
%%sql
SELECT * From User

 * sqlite:///courses.db
Done.


id,name,email
1,Jane,jane@uvic.ca
2,Manke,manke@uvic.ca
3,Ryan,ryan@uvic.ca


In [5]:
%%sql
INSERT INTO Course(text) VALUES ('Scientific_Computing');
INSERT into Course(text) VALUES ('Advanced_PDE');
INSERT into Course(text) VALUES ('Numerical_Finance')

 * sqlite:///courses.db
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [6]:
%%sql
SELECT * From Course

 * sqlite:///courses.db
Done.


id,text
1,Scientific_Computing
2,Advanced_PDE
3,Numerical_Finance


In [7]:
%%sql
INSERT INTO Member(user_id, course_id,role) VALUES (1,1,1);
INSERT INTO Member(user_id, course_id,role) VALUES (2,1,0);
INSERT INTO Member(user_id, course_id,role) VALUES (3,1,0);

INSERT INTO Member(user_id, course_id,role) VALUES (1,2,0);
INSERT INTO Member(user_id, course_id,role) VALUES (2,2,1);

INSERT INTO Member(user_id, course_id,role) VALUES (2,3,1);
INSERT INTO Member(user_id, course_id,role) VALUES (3,3,0)

 * sqlite:///courses.db
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [8]:
%%sql
SELECT * From Member

 * sqlite:///courses.db
Done.


user_id,course_id,role
1,1,1
2,1,0
3,1,0
1,2,0
2,2,1
2,3,1
3,3,0


Now using the previous three tables, we make a new one using "JOIN". Our aim is toget the name, role and title of the course.

In [9]:
%%sql
SELECT User.name,Member.role,Course.text
From User JOIN Member JOIN Course
ON Member.user_id = User.id AND Member.course_id = Course.id
ORDER BY Course.text, Member.role DESC, User.name

 * sqlite:///courses.db
Done.


name,role,text
Manke,1,Advanced_PDE
Jane,0,Advanced_PDE
Manke,1,Numerical_Finance
Ryan,0,Numerical_Finance
Jane,1,Scientific_Computing
Manke,0,Scientific_Computing
Ryan,0,Scientific_Computing


Here, "DESC" stands for descending (course first and role).

Lets implement in python now. But first, we take a look at the subset of the dataset, we will be working on, which is an array of array.
<code>
  [
    "Charley",
    "si110",
    1
  ],
  [
    "Mea",
    "si110",
    0
  ],
  [
    "Hattie",
    "si110",
    0
  ]  
</code>


In [15]:
import json
import sqlite3

conn = sqlite3.connect('rosterdb.sqlite')
cur = conn.cursor()

# Do some setup
cur.executescript('''
DROP TABLE IF EXISTS User;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Course;

CREATE TABLE User (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name   TEXT UNIQUE
);

CREATE TABLE Course (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    text  TEXT UNIQUE
);

CREATE TABLE Member (
    user_id     INTEGER,
    course_id   INTEGER,
    role        INTEGER,
    PRIMARY KEY (user_id, course_id)
)
''')

fname = input('Enter file name: ')
if len(fname) < 1:
    fname = 'roster_data_sample.json'

# [
#   [ "Charley", "si110", 1 ],
#   [ "Mea", "si110", 0 ],

str_data = open(fname).read()
json_data = json.loads(str_data)

for entry in json_data:

    name = entry[0];
    text = entry[1];

    print((name, text))

    cur.execute('''INSERT OR IGNORE INTO User (name)
        VALUES ( ? )''', ( name, ) )
    cur.execute('SELECT id FROM User WHERE name = ? ', (name, ))
    user_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Course (text)
        VALUES ( ? )''', ( text, ) )
    cur.execute('SELECT id FROM Course WHERE text = ? ', (text, ))
    course_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Member
        (user_id, course_id) VALUES ( ?, ? )''',
        ( user_id, course_id ) )

    conn.commit()

Enter file name: roster_data_sample.json
('Charley', 'si110')
('Mea', 'si110')
('Hattie', 'si110')
('Lyena', 'si110')
('Keziah', 'si110')
('Ellyce', 'si110')
('Thalia', 'si110')
('Meabh', 'si110')
('Aria', 'si110')
('Reena', 'si110')
('Ioannis', 'si110')
('Reily', 'si110')
('Sidharth', 'si110')
('Keiara', 'si110')
('Yann', 'si110')
('Marykate', 'si110')
('Dylan', 'si110')
('Kiran', 'si110')
('Faizaan', 'si110')
('Aneshia', 'si110')
('Kamron', 'si110')
('Allen', 'si110')
('Marshall', 'si110')
('Rosa', 'si106')
('Nora', 'si106')
('Mairin', 'si106')
('Zendel', 'si106')
('Honie', 'si106')
('Betsy', 'si106')
('Davie', 'si106')
('Larissa', 'si106')
('Shaurya', 'si106')
('Shania', 'si106')
('Sorcha', 'si106')
('Jeanna', 'si106')
('Temba', 'si106')
('Buse', 'si106')
('Mohammed', 'si106')
('Kayah', 'si106')
('Kareena', 'si106')
('Dineo', 'si106')
('Philippa', 'si106')
('Lia', 'si206')
('Sharlyn', 'si206')
('Linton', 'si206')
('Temilade', 'si206')
('Areez', 'si206')
('MacCartney', 'si206')
('Abu

This application will read roster data in JSON format, parse the file, and then produce an SQLite database that contains a User, Course, and Member table and populate the tables from the data file.

In [19]:
import json
import sqlite3

conn = sqlite3.connect('rosterdb1.sqlite')
cur = conn.cursor()

# Do some setup
cur.executescript('''
DROP TABLE IF EXISTS User;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Course;

CREATE TABLE User (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name   TEXT UNIQUE
);

CREATE TABLE Course (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    title  TEXT UNIQUE
);

CREATE TABLE Member (
    user_id     INTEGER,
    course_id   INTEGER,
    role        INTEGER,
    PRIMARY KEY (user_id, course_id)
)
''')

fname = input('Enter file name: ')
if ( len(fname) < 1 ) : fname = 'roster_data.json'

# [
#   [ "Charley", "si110", 1 ],
#   [ "Mea", "si110", 0 ],

str_data = open(fname).read()
json_data = json.loads(str_data)

for entry in json_data:

    name = entry[0];
    title = entry[1];
    role = entry[2];
    print (name, title, role)

    cur.execute('''INSERT OR IGNORE INTO User (name) 
        VALUES ( ? )''', ( name, ) )
    cur.execute('SELECT id FROM User WHERE name = ? ', (name, ))
    user_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Course (title) 
        VALUES ( ? )''', ( title, ) )
    cur.execute('SELECT id FROM Course WHERE title = ? ', (title, ))
    course_id = cur.fetchone()[0]

    
    
    cur.execute('''INSERT OR REPLACE INTO Member
        (user_id, course_id, role) VALUES ( ?, ?, ? )''', 
        ( user_id, course_id, role ) )

    conn.commit()

Enter file name: roster_data.json
Thomson si110 1
Dharam si110 0
Aslam si110 0
Melville si110 0
Makenzie si110 0
Johanna si110 0
Nickson si110 0
Bentley si110 0
Freia si110 0
Johnjay si110 0
Brody si110 0
Aaryan si110 0
Caitlynn si110 0
Malia si110 0
Prince si110 0
Zendel si110 0
Nicki si110 0
Lily si106 1
Fauzaan si106 0
Mohamad si106 0
Ekhlass si106 0
Priscillia si106 0
Madeline si106 0
Reo si106 0
Tabbitha si106 0
Alekzander si106 0
Indiana si106 0
Coben si106 0
Lilliarna si106 0
Morgan si106 0
Nidba si106 0
Harlie si106 0
Ariah si106 0
Dallace si106 0
Safi si106 0
Cesar si106 0
Berlin si106 0
Lorenz si106 0
Jillian si106 0
Nola si106 0
Sung si106 0
Linton si106 0
Sameera si106 0
Kaylyn si106 0
Codie si106 0
Craig si106 0
Julietta si106 0
Ayrton si106 0
Aleena si106 0
Joojo si106 0
Moosa si106 0
Kadin si106 0
Miya si106 0
Rylan si106 0
Kornelija si106 0
Eagann si106 0
Brendyn si106 0
Leona si206 1
Diarmuid si206 0
Koden si206 0
Nana si206 0
Keir si206 0
Rubhan si206 0
Deano si206 0
